# Import Libraries

In [1]:
import os
import glob
import pandas as pd
import numpy as np

# Download

In [2]:
basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

# Title Basics

In [3]:
basics = pd.read_csv(basics_url, sep = '\t', low_memory = False)

In [4]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


## Process Data

### Replace null values (\N) with NAN


In [5]:
basics.replace({'\\N':np.nan}, inplace = True)

In [6]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


### Eliminate movies that are null for 'runtimeMinutes' and 'genres'

In [7]:
basics = basics.dropna(subset = ['runtimeMinutes', 'genres'])

### Keep only titleType==Movie

In [8]:
basics = basics.loc[basics['titleType'] == 'movie']
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"


### Keep startYear 2000-2022

In [9]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374579 entries, 8 to 9504554
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          374579 non-null  object
 1   titleType       374579 non-null  object
 2   primaryTitle    374579 non-null  object
 3   originalTitle   374579 non-null  object
 4   isAdult         374579 non-null  object
 5   startYear       368250 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  374579 non-null  object
 8   genres          374579 non-null  object
dtypes: object(9)
memory usage: 28.6+ MB


The type of 'startYear' is object.

In [10]:
basics = basics.dropna(subset = ['startYear'])
basics['startYear'] = basics['startYear'].astype('int64')
basics = basics.loc[(basics['startYear']>= 2000) & (basics['startYear']<= 2022)]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,133,Documentary
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
66336,tt0067683,movie,Workers '71: Nothing About Us Without Us,Robotnicy 1971 - Nic o nas bez nas,0,2006,NaN,47,Documentary
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama


### Eliminate movies that include "Documentary" in genre

In [11]:
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77964,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [12]:
## Make the Data folder if it doesn't already exist
os.makedirs('Data',exist_ok=True)

In [13]:
# Confirm folder created
os.listdir("Data/")

['title_basics.csv.gz', 'title_ratings.csv.gz']

In [14]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [15]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [16]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145983 entries, 0 to 145982
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          145983 non-null  object 
 1   titleType       145983 non-null  object 
 2   primaryTitle    145983 non-null  object 
 3   originalTitle   145983 non-null  object 
 4   isAdult         145983 non-null  int64  
 5   startYear       145983 non-null  int64  
 6   endYear         0 non-null       float64
 7   runtimeMinutes  145983 non-null  int64  
 8   genres          145983 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 10.0+ MB


# Title Ratings

In [17]:
ratings = pd.read_csv(ratings_url, sep = '\t', low_memory = False)

In [18]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1933
1,tt0000002,5.8,262
2,tt0000003,6.5,1755
3,tt0000004,5.6,177
4,tt0000005,6.2,2565


## Replace "\N" with np.nan (if any)

In [19]:
ratings.replace({'\\N':np.nan}, inplace = True)

In [20]:
## Save current dataframe to file.
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [21]:
# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1933
1,tt0000002,5.8,262
2,tt0000003,6.5,1755
3,tt0000004,5.6,177
4,tt0000005,6.2,2565


In [22]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1263864 entries, 0 to 1263863
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1263864 non-null  object 
 1   averageRating  1263864 non-null  float64
 2   numVotes       1263864 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.9+ MB


# Title Akas

In [23]:
chunk_num = 1
akas_reader = pd.read_csv(akas_url, sep='\t', low_memory=False, chunksize=100_000)

In [24]:
for temp_akas in akas_reader:
        #### COMBINED WORKFLOW FROM ABOVE
    ## Replace "\N" with np.nan
    temp_akas.replace({'\\N':np.nan},inplace=True)
    
    ## keep only US movies
    temp_akas = temp_akas.loc[temp_akas['region']=='US']

    ### Saving chunk to disk
    fname= f'Data/title_akas_chunk_{chunk_num:03d}.csv.gz'
    temp_akas.to_csv(fname, compression='gzip')
    print(f"- Saved {fname}")
    
    ## increment chunk_num    
    chunk_num+=1
## Closing the reader now that we are done looping through the file
akas_reader.close()

- Saved Data/title_akas_chunk_001.csv.gz
- Saved Data/title_akas_chunk_002.csv.gz
- Saved Data/title_akas_chunk_003.csv.gz
- Saved Data/title_akas_chunk_004.csv.gz
- Saved Data/title_akas_chunk_005.csv.gz
- Saved Data/title_akas_chunk_006.csv.gz
- Saved Data/title_akas_chunk_007.csv.gz
- Saved Data/title_akas_chunk_008.csv.gz
- Saved Data/title_akas_chunk_009.csv.gz
- Saved Data/title_akas_chunk_010.csv.gz
- Saved Data/title_akas_chunk_011.csv.gz
- Saved Data/title_akas_chunk_012.csv.gz
- Saved Data/title_akas_chunk_013.csv.gz
- Saved Data/title_akas_chunk_014.csv.gz
- Saved Data/title_akas_chunk_015.csv.gz
- Saved Data/title_akas_chunk_016.csv.gz
- Saved Data/title_akas_chunk_017.csv.gz
- Saved Data/title_akas_chunk_018.csv.gz
- Saved Data/title_akas_chunk_019.csv.gz
- Saved Data/title_akas_chunk_020.csv.gz
- Saved Data/title_akas_chunk_021.csv.gz
- Saved Data/title_akas_chunk_022.csv.gz
- Saved Data/title_akas_chunk_023.csv.gz
- Saved Data/title_akas_chunk_024.csv.gz
- Saved Data/tit

- Saved Data/title_akas_chunk_201.csv.gz
- Saved Data/title_akas_chunk_202.csv.gz
- Saved Data/title_akas_chunk_203.csv.gz
- Saved Data/title_akas_chunk_204.csv.gz
- Saved Data/title_akas_chunk_205.csv.gz
- Saved Data/title_akas_chunk_206.csv.gz
- Saved Data/title_akas_chunk_207.csv.gz
- Saved Data/title_akas_chunk_208.csv.gz
- Saved Data/title_akas_chunk_209.csv.gz
- Saved Data/title_akas_chunk_210.csv.gz
- Saved Data/title_akas_chunk_211.csv.gz
- Saved Data/title_akas_chunk_212.csv.gz
- Saved Data/title_akas_chunk_213.csv.gz
- Saved Data/title_akas_chunk_214.csv.gz
- Saved Data/title_akas_chunk_215.csv.gz
- Saved Data/title_akas_chunk_216.csv.gz
- Saved Data/title_akas_chunk_217.csv.gz
- Saved Data/title_akas_chunk_218.csv.gz
- Saved Data/title_akas_chunk_219.csv.gz
- Saved Data/title_akas_chunk_220.csv.gz
- Saved Data/title_akas_chunk_221.csv.gz
- Saved Data/title_akas_chunk_222.csv.gz
- Saved Data/title_akas_chunk_223.csv.gz
- Saved Data/title_akas_chunk_224.csv.gz
- Saved Data/tit

Tip: if we use the ":03d" format code when inserting the chunk number using an f-string, it will add 2 leading 0's, so the first file will be numbered 001 instead of 1. This will be helpful when viewing the files in your file explorer or on GitHub.

In [25]:
q = "Data/title_akas_chunk*.csv.gz"
chunked_files = glob.glob(q)
# Showing the first 5
chunked_files[:5]

['Data\\title_akas_chunk_001.csv.gz',
 'Data\\title_akas_chunk_002.csv.gz',
 'Data\\title_akas_chunk_003.csv.gz',
 'Data\\title_akas_chunk_004.csv.gz',
 'Data\\title_akas_chunk_005.csv.gz']

In [26]:
## Loading all files as df and appending to a list
akas_list = []
for file in chunked_files:
    temp_akas = pd.read_csv(file, index_col=0)
    akas_list.append(temp_akas)
    
## Concatenating the list of dfs into 1 combined
akas_combined = pd.concat(akas_list)
akas_combined.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [27]:
#Include only movies that were released in the United States
akas_combined = akas_combined.loc[akas_combined['region'] =='US']

In [28]:
## Saving the final combined dataframe
final_fname ='Data/title_akas_combined.csv.gz'
akas_combined.to_csv(final_fname, compression='gzip', index=False)

In [29]:
# Open saved file and preview again
akas_combined = pd.read_csv(final_fname)
akas_combined.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [30]:
akas_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1402139 entries, 0 to 1402138
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1402139 non-null  object 
 1   ordering         1402139 non-null  int64  
 2   title            1402139 non-null  object 
 3   region           1402139 non-null  object 
 4   language         3762 non-null     object 
 5   types            968985 non-null   object 
 6   attributes       45647 non-null    object 
 7   isOriginalTitle  1400794 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 85.6+ MB


# Filtering one dataframe based on another

In [31]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas_combined['titleId'])
keepers

0          True
1          True
2          True
3         False
4          True
          ...  
145978     True
145979     True
145980    False
145981     True
145982    False
Name: tconst, Length: 145983, dtype: bool

In [32]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
7,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
145975,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
145977,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
145978,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
145979,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [33]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)